In [1]:
import torch
import torch.nn as nn
from model import*
from quant_utils import *
from train import eval_model, finetune
from data import get_cifar10_dataloader
import args

In [2]:
import torch_npu
device = torch.device("npu")

[W OperatorEntry.cpp:121] Warning: Overriding a previously registered kernel for the same operator and the same dispatch key
  operator: aten::_has_compatible_shallow_copy_type(Tensor self, Tensor from) -> (bool)
    registered at /usr1/pytorch/build/aten/src/ATen/RegisterSchema.cpp:20
  dispatch key: Math
  previous kernel: registered at /usr1/pytorch/build/aten/src/ATen/RegisterMath.cpp:5686
       new kernel: registered at /usr1/workspace/FPTA_Daily_Plugin_open_date/Plugin/torch_npu/csrc/aten/ops/HasCompatibleShallowCopyType.cpp:37 (function registerKernel)


In [3]:
train_loader = get_cifar10_dataloader()
test_loader = get_cifar10_dataloader(train=False)
vit = CAFIA_Transformer(args.QVIT)
path = '/home/ma-user/work/Vision-Transformer-ViT/output/vit/0.9853000044822693.pt'
load_weight_for_vit(vit, path)

_IncompatibleKeys(missing_keys=['vit.transformer.encoder_layers.0.attn.query.activation_range', 'vit.transformer.encoder_layers.0.attn.query.weight_range', 'vit.transformer.encoder_layers.0.attn.key.activation_range', 'vit.transformer.encoder_layers.0.attn.key.weight_range', 'vit.transformer.encoder_layers.0.attn.value.activation_range', 'vit.transformer.encoder_layers.0.attn.value.weight_range', 'vit.transformer.encoder_layers.0.attn.out.activation_range', 'vit.transformer.encoder_layers.0.attn.out.weight_range', 'vit.transformer.encoder_layers.0.mlp.fc1.activation_range', 'vit.transformer.encoder_layers.0.mlp.fc1.weight_range', 'vit.transformer.encoder_layers.0.mlp.fc2.activation_range', 'vit.transformer.encoder_layers.0.mlp.fc2.weight_range', 'vit.transformer.encoder_layers.1.attn.query.activation_range', 'vit.transformer.encoder_layers.1.attn.query.weight_range', 'vit.transformer.encoder_layers.1.attn.key.activation_range', 'vit.transformer.encoder_layers.1.attn.key.weight_range', 

In [4]:
w, a = 5, 8

In [7]:
set_hwq(vit, True)
idx, strategy = get_single_prec_quant_strategy(vit, w, a)
set_mixed_precision(vit, idx, strategy)
finetune(vit, train_loader, device, lr=1e-4)
eval_model(vit, test_loader, device)

INFO:root:**************************** start to finetune 1 turns*******************************
Iteration:   0%|          | 0/1563 [00:00<?, ?it/s][W OpCommand.cpp:82] Warning: [Check][offset] Check input storage_offset[%ld] = 0 failed, result is untrustworthy768 (function operator())
INFO:root:Epoch:0 batch_loss:2.509468
Iteration: 100%|██████████| 1563/1563 [08:40<00:00,  3.00it/s]
INFO:root:**************************** start to evaluate *******************************
Iteration: 100%|██████████| 313/313 [00:56<00:00,  5.56it/s]
INFO:root:Accuracy: 0.963200


In [6]:
set_hwq(vit, False)
idx, strategy = get_single_prec_quant_strategy(vit, w, a)
set_mixed_precision(vit, idx, strategy)
# finetune(vit, train_loader, device, lr=1e-4)
eval_model(vit, test_loader, device)

INFO:root:**************************** start to evaluate *******************************
Iteration: 100%|██████████| 313/313 [00:54<00:00,  5.75it/s]
INFO:root:Accuracy: 0.493100
